CAPÍTULO 13 - Modelos Lineares Generalizados
-


13.1 Introdução

Nem toda variável de resposta é contínua, portanto nem sempre uma regressão linear será o modelo correto em todas as ocasiões.  
Uma classe geral de modelos chamada GLM (Generalizad Linear Models, ou modelos lineares generalizados) pode dar conta desse tipo de dado.

13.2 Regressão logística

Regressão logística é muito usada para uma variável de resposta binária.

In [36]:
import pandas as pd

acs = pd.read_csv('../../data/acs_ny.csv')
print(acs.columns)

Index(['Acres', 'FamilyIncome', 'FamilyType', 'NumBedrooms', 'NumChildren',
       'NumPeople', 'NumRooms', 'NumUnits', 'NumVehicles', 'NumWorkers',
       'OwnRent', 'YearBuilt', 'HouseCosts', 'ElectricBill', 'FoodStamp',
       'HeatingFuel', 'Insurance', 'Language'],
      dtype='object')


In [37]:
print(acs.head())

  Acres  FamilyIncome   FamilyType  NumBedrooms  NumChildren  NumPeople  \
0  1-10           150      Married            4            1          3   
1  1-10           180  Female Head            3            2          4   
2  1-10           280  Female Head            4            0          2   
3  1-10           330  Female Head            2            1          2   
4  1-10           330    Male Head            3            1          2   

   NumRooms         NumUnits  NumVehicles  NumWorkers   OwnRent    YearBuilt  \
0         9  Single detached            1           0  Mortgage    1950-1959   
1         6  Single detached            2           0    Rented  Before 1939   
2         8  Single detached            3           1  Mortgage    2000-2004   
3         4  Single detached            1           0    Rented    1950-1959   
4         5  Single attached            1           0  Mortgage  Before 1939   

   HouseCosts  ElectricBill FoodStamp HeatingFuel  Insurance        

Para modelar esse dados, devemos inicialmente criar uma variável de resposta binária. Nesse caso a variável FamilyIncome.

In [38]:
acs['ge150k'] = pd.cut(acs['FamilyIncome'], #refencia a coluna que será usada
                       [0,150000, acs['FamilyIncome'].max()], #cria dois intervalos, de 0 à 150000 e de 150000 até o valor máximo da coluna
                       labels=[0, 1]) #atribuí um rotulo a cada intervalo 0 = renda até 150000, 1 = renda acima de 150000

acs['ge150k_i'] = acs['ge150k'].astype(int) #converte os rotulos para valores inteiros, se não fizesemos isso a coluna sería category

print(acs['ge150k_i'].value_counts())

ge150k_i
0    18294
1     4451
Name: count, dtype: int64


Ao fazer isso, criamos uma variável binária

In [39]:
acs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22745 entries, 0 to 22744
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Acres         22745 non-null  object  
 1   FamilyIncome  22745 non-null  int64   
 2   FamilyType    22745 non-null  object  
 3   NumBedrooms   22745 non-null  int64   
 4   NumChildren   22745 non-null  int64   
 5   NumPeople     22745 non-null  int64   
 6   NumRooms      22745 non-null  int64   
 7   NumUnits      22745 non-null  object  
 8   NumVehicles   22745 non-null  int64   
 9   NumWorkers    22745 non-null  int64   
 10  OwnRent       22745 non-null  object  
 11  YearBuilt     22745 non-null  object  
 12  HouseCosts    22745 non-null  int64   
 13  ElectricBill  22745 non-null  int64   
 14  FoodStamp     22745 non-null  object  
 15  HeatingFuel   22722 non-null  object  
 16  Insurance     22745 non-null  int64   
 17  Language      22745 non-null  object  
 18  ge150k

13.2.1 Usando a statsmodels

Para fazer a regressão logística, podemos usar a regressão logit. A sintaxe é a mesma usada para egressão linear.

In [41]:
import statsmodels.formula.api as smf 

model = smf.logit('ge150k_i ~ HouseCosts + NumWorkers + OwnRent + NumBedrooms + FamilyType',
                  data=acs)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.391651
         Iterations 7


In [42]:
print(results.summary())

                           Logit Regression Results                           
Dep. Variable:               ge150k_i   No. Observations:                22745
Model:                          Logit   Df Residuals:                    22737
Method:                           MLE   Df Model:                            7
Date:                Sat, 29 Jun 2024   Pseudo R-squ.:                  0.2078
Time:                        12:36:45   Log-Likelihood:                -8908.1
converged:                       True   LL-Null:                       -11244.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -5.8081      0.120    -48.456      0.000      -6.043      -5.573
OwnRent[T.Outright]         1.8276      0.208      8.782      0.000       1.420       2.236


Interpretar os dados de uma regessão logística não é tão simples quanto interpretar os dados de uma regressão linear. Em uma regressão logística e outros modelos lineares generalizados, há uma transformação na forma de uma função de link que deve ser desfeita para interpretar o resultado.

Para interpretar o nosso modelo logístico precisamos exponenciar os resultados.

In [43]:
import numpy as np

odds_ratios = np.exp(results.params)
print(odds_ratios)

Intercept                  0.003003
OwnRent[T.Outright]        6.219147
OwnRent[T.Rented]          0.416310
FamilyType[T.Male Head]    1.332901
FamilyType[T.Married]      4.005636
HouseCosts                 1.000731
NumWorkers                 1.799117
NumBedrooms                1.266852
dtype: float64


Os valores estão interpretados como razões de probabilidades. Podemos pensar em uma razão de possibilidades como quantas vezes os resultado será "provável".   
Exemplo: podemos dizer que a cada incremento de unidade em NumBedrooms as chances de FamilyIncome ser maior que 150000 aumenta em 1,27 vezes.  
Uma interpretasção semelhante pode ser usada em variáveis categorizadas.

In [45]:
#os três valores de ownrent
print(acs.OwnRent.unique())

['Mortgage' 'Rented' 'Outright']


Um exemplo de interpretação seria dizer que as chaces de familyIncome ser maior que 150000 aumenta 1,82 vezes quando a casa é própria em comparação com estar financiada.